<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5_1300ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jan 11 07:46:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 110.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
import pandas as pd

data = pd.read_csv('/content/gdrive/MyDrive/sentences_data.csv')
data

,English,Thai
0,There are several agricultural residues in the...,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...
1,"At present, biomass is being used in the small...",ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก
2,It is an effective way to eliminate agricultur...,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...
3,"Recently, there are many incentives from the g...",เมื่อเร็ว ๆ นี้มีแรงจูงใจมากมายจากรัฐบาลสำหรับ...
4,"For examples, these electricity producers are ...",ตัวอย่างเช่น ผู้ผลิตไฟฟ้าเหล่านี้ได้รับอนุญาตใ...
...,...,...
1288,But the laws require the energy management wor...,แต่กฎหมายกำหนดให้คณะทำงานด้านการจัดการพลังงานต...
1289,The objective of the research is to develop me...,วัตถุประสงค์ของงานวิจัยคือการพัฒนาวิธีการในการ...
1290,"In this work, the control charts for individua...",ในงานนี้แผนภูมิควบคุมสำหรับข้อมูลเชิงเดียวและค...
1291,The criterion used in determining the achievem...,เกณฑ์ที่ใช้ในการพิจารณาผลสัมฤทธิ์จากมาตรการฯ ค...


In [6]:
import csv

with open('/content/dataset.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['prefix', 'input_text', 'target_text'])
  for i in range(len(data)):
    writer.writerow(['translate thai to english', data['Thai'][i], data['English'][i]])

pd.read_csv('/content/dataset.csv')

,prefix,input_text,target_text
0,translate thai to english,มีวัสดุเหลือใช้ทางการเกษตรมากมายในประเทศที่สาม...,There are several agricultural residues in the...
1,translate thai to english,ปัจจุบันมีการใช้ชีวมวลในการผลิตไฟฟ้าขนาดเล็ก,"At present, biomass is being used in the small..."
2,translate thai to english,เป็นวิธีที่มีประสิทธิภาพในการกำจัดของเสียทางกา...,It is an effective way to eliminate agricultur...
3,translate thai to english,เมื่อเร็ว ๆ นี้มีแรงจูงใจมากมายจากรัฐบาลสำหรับ...,"Recently, there are many incentives from the g..."
4,translate thai to english,ตัวอย่างเช่น ผู้ผลิตไฟฟ้าเหล่านี้ได้รับอนุญาตใ...,"For examples, these electricity producers are ..."
...,...,...,...
1288,translate thai to english,แต่กฎหมายกำหนดให้คณะทำงานด้านการจัดการพลังงานต...,But the laws require the energy management wor...
1289,translate thai to english,วัตถุประสงค์ของงานวิจัยคือการพัฒนาวิธีการในการ...,The objective of the research is to develop me...
1290,translate thai to english,ในงานนี้แผนภูมิควบคุมสำหรับข้อมูลเชิงเดียวและค...,"In this work, the control charts for individua..."
1291,translate thai to english,เกณฑ์ที่ใช้ในการพิจารณาผลสัมฤทธิ์จากมาตรการฯ ค...,The criterion used in determining the achievem...


In [7]:
!rm -rf '/content/train.csv'
!rm -rf '/content/eval.csv'
!rm -rf '/content/test.csv'

from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/dataset.csv')
train_data, eval_test = train_test_split(data, train_size=0.9, shuffle=True)
train_data.to_csv('/content/train.csv', index=False)
eval_data, test_data = train_test_split(eval_test, train_size=0.9, shuffle=True)
eval_data.to_csv('/content/eval.csv', index=False)
test_data.to_csv('/content/test.csv', index=False)

In [8]:
pd.read_csv('/content/train.csv')

,prefix,input_text,target_text
0,translate thai to english,การวิจัยครั้งนี้มีวัตถุประสงค์,The objectives of this research were
1,translate thai to english,และเป็นไปไม่ได้ที่จะวิจัยต้นแบบในทุกช่วงขนาดมอ...,And it is not possible to research prototypes ...
2,translate thai to english,นอกจากนี้เกษตรกรและพ่อค้าแม่ค้าในตลาดสดให้ความ...,"Moreover, farmers and vendors are more concern..."
3,translate thai to english,ระบบป้องกันการจ่ายไฟย้อนกลับสู่ระบบสาธารณูปโภค...,The protection system for reverse power supply...
4,translate thai to english,เก็บข้อมูลระหว่างเดือนสิงหาคมถึงธันวาคม พ.ศ. 2...,Data were collected between August and Decembe...
...,...,...,...
1158,translate thai to english,เมื่อเกลียวหมุนสามารถลำเลียงลูกบอลพร้อมทั้งทำค...,"While the screw conveyer rotates, the screw pl..."
1159,translate thai to english,ซึ่งเป็นกลุ่มตัวอย่างจำนวน 385 คน โดยใช้แบบสอบ...,Which is a sample of 385 people by using a que...
1160,translate thai to english,โดยเงื่อนไขที่เหมาะสมที่สุดของการอบแห้งว่านหาง...,The optimization condition of dried aloe vera ...
1161,translate thai to english,การศึกษาครั้งนี้สามารถประยุกต์ใช้งานข้อมูลความ...,"According to the demonstrated result, the use ..."


In [9]:
pd.read_csv('/content/eval.csv')

,prefix,input_text,target_text
0,translate thai to english,สาธารณรัฐประชาธิปไตยประชาชนลาว.,"People's Democratic Republic, Lao People's Dem..."
1,translate thai to english,สุดท้ายเป็นการประเมินประสิทธิภาพของระบบตรวจวัด...,"Finally, it was to the evaluation of the measu..."
2,translate thai to english,โดยใช้น้ำยางพาราเป็นตัวประสาน ผสมกับน้ำ และสาร...,"By using Para rubber as a binder, mixed with w..."
3,translate thai to english,งานวิจัยนี้เป็นการศึกษาเพื่อประเมินความเสี่ยงข...,This research is a study to assess the risk of...
4,translate thai to english,ประสิทธิภาพการกำจัดของแข็งระเหยทั้งหมดอยู่ระหว...,The total volatile solids removal efficiency w...
...,...,...,...
112,translate thai to english,ในการทดลอง กระเป๋าเก็บพลังงานสามารถจ่ายพลังงาน...,"On the experiment, energy harvest backpack can..."
113,translate thai to english,การหลั่งไหลของรถยนต์ไฟฟ้าทั่วโลกมีแนวโน้มเติบโ...,The global influx of electric vehicles is like...
114,translate thai to english,การสุ่มตัวอย่างพื้นที่ดำเนินการเป็นพื้นฐานก่อน...,Area sampling was performed as a baseline befo...
115,translate thai to english,ส่วนที่หนึ่งจะทดสอบวัดระยะทางกับ 5 วัตถุ,First part was for those five materials.


In [10]:
pd.read_csv('/content/test.csv')

,prefix,input_text,target_text
0,translate thai to english,บทความวิชาการนี้ นำเสนอการศึกษางานวิจัยเกี่ยวก...,This academic article presents a research stud...
1,translate thai to english,ด้านวัสดุ คือ วัสดุในการผลิต ความทนทานของวัสดุ...,The material part is the production material d...
2,translate thai to english,ระบบคุณภาพ ISO 9001 สำหรับการควบคุมคุณภาพด้วยเ...,ISO 9001 Quality System for Quality Control wi...
3,translate thai to english,งานวิจัยนี้มีวัตถุประสงค์เพื่อออกแบบและศึกษาปร...,The objective of this research is to design an...
4,translate thai to english,ระดับความเอียงที่เหมาะสมคือ 20 องศา ได้ปริมาณน...,The suitable inclination level was 20 degrees ...
5,translate thai to english,อาร์เรย์มุมฉาก L18 เพื่อลดจำนวนการทดลองและวิเค...,Orthogonal array L18 to reduce the number of e...
6,translate thai to english,ระบบมีการติดตั้งอุปกรณ์ตรวจวัดอุณหภูมิและความช...,The system has installed a temperature and hum...
7,translate thai to english,นอกจากนี้ระยะเวลาเป็นสิ่งที่สำคัญในการออกแบบห้...,"So that, designing of the mobile calibration l..."
8,translate thai to english,เครื่องบำบัดน้ำเสียด้วยโอโซนในสภาวะความดันสูงถ...,Ozone Wastewater Treatment under High Pressure...
9,translate thai to english,จากการทดสอบเบื้องต้นพบว่า การสั่นสะเทือนของวัต...,The initial testing shows that the vibration o...


In [11]:
import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/train.csv').astype(str)
eval_df = pd.read_csv('/content/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.max_seq_length = 96
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.learning_rate = 1e-3
model_args.num_train_epochs = 32
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [12]:
# !wandb login --relogin

In [13]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/1163 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/32 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

...

Running Epoch 30 of 32:   0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

Running Epoch 31 of 32:   0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/117 [00:00<?, ?it/s]

(4672,
 {'global_step': [146,
   292,
   438,
   584,
   730,
   876,
   1022,
   1168,
   1314,
   1460,
   1606,
   1752,
   1898,
   2044,
   2190,
   2336,
   2482,
   2628,
   2774,
   2920,
   3066,
   3212,
   3358,
   3504,
   3650,
   3796,
   3942,
   4088,
   4234,
   4380,
   4526,
   4672],
  'eval_loss': [5.133438936869303,
   3.3133545875549317,
   2.789562765757243,
   2.609515150388082,
   2.485822002092997,
   2.715831232070923,
   6.0554144541422525,
   4.037387816111247,
   4.020973698298136,
   4.072049919764201,
   4.09475998878479,
   4.319702943166097,
   4.459951766331991,
   4.896862570444743,
   4.950709422429402,
   5.0829916795094805,
   5.505173142751058,
   5.606709829966227,
   5.556337324778239,
   5.803031063079834,
   6.332778549194336,
   6.234726015726725,
   6.400146230061849,
   6.636016972859701,
   6.638685512542724,
   6.937526925404867,
   6.566991456349691,
   6.826807498931885,
   6.152999305725098,
   5.078069114685059,
   4.65150671005249,

In [14]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/117 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [16]:
valid = pd.read_csv("/content/eval.csv").astype(str)

# thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
# to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [17]:
english_preds = model.predict(to_english)

then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

Generating outputs:   0%|          | 0/15 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/117 [00:00<?, ?it/s]

Thai to English:  1.1040947174675297


In [24]:
from simpletransformers.t5 import T5Model

# model = T5Model("mt5", "/content/outputs")
to_predict = ['การสร้างเครื่องแปลภาษาเป็นโจทย์ classic ของ NLP และสามารถนำไปใช้ประโยชน์ได้โดยตรงโดยตัวของมันเอง โจทย์นี้จัดว่าเป็นโจทย์ที่ยากที่สุดของ NLP เพราะต้องอาศัยความเข้าใจของภาษาในระดับที่ลึกซึ้งทำให้โมเดลที่ต้องใช้ต้องมีความซับซ้อนตามไปด้วย']
to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
predictions

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3704: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/8 [00:00<?, ?it/s]

['The purpose of this research is to study and development of bamboo furniture in the type of sofa set.',
 'The purpose of this research is to study and development of bamboo furniture in the type of sofa set.',
 'The average of FA concentrations in the anatomy laboratory room was 0.155 ppm (0.126 –0.898).',
 'The highest R2 is 0.9925 and the lowest and RMSE are 0.0001 and 0.0044, respectively.',
 'The purpose of this research is to study the development of briquette charcoal Analysis of sugarcane leaves and bagasse to community.',
 'The purpose of this research is to study the development of briquette charcoal Analysis of sugarcane leaves and bagasse to community.',
 'The highest R2 is 0.9925 and the lowest and RMSE are 0.0001 and 0.0044, respectively.',
 'The purpose of this research is to study and development of bamboo furniture in the type of sofa set.']

In [23]:
to_predict[0].split()

['การสร้างเครื่องแปลภาษาเป็นโจทย์',
 'classic',
 'ของ',
 'NLP',
 'และสามารถนำไปใช้ประโยชน์ได้โดยตรงโดยตัวของมันเอง',
 'โจทย์นี้จัดว่าเป็นโจทย์ที่ยากที่สุดของ',
 'NLP',
 'เพราะต้องอาศัยความเข้าใจของภาษาในระดับที่ลึกซึ้งทำให้โมเดลที่ต้องใช้ต้องมีความซับซ้อนตามไปด้วย']

In [ ]:
# test_data['input_text']

In [ ]:
# with open('/content/predict.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['ref_th', 'ref_eng', 'predict'])
#   for i in range(len(test_data)):
#     writer.writerow([test_data['input_text'][i], test_data['target_text'][i], predictions[i]])
# pd.read_csv('/content/predict.csv')